<a href="https://colab.research.google.com/github/abhijain864/VTON-Intern/blob/main/newlook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Categories are<br> - For_Men - t-shirts , shirts , knitwear , sweatshirts , jeans , trousers
# -For_Women - t-shirts , jeans , trousers , shirts

# Remarks
# Note that in men-shirts category, many images with open shirt are there.
# In men-trousers, some images have upper-wear and lower-wear of the same color
# In women tops-shirt-blouses, some images have upper-wear and lower-wear of the same color
# In women tops-t-shirts, many upper body images are not fully upper body

In [ ]:
import time
import requests
from requests.exceptions import ConnectionError
import os
from bs4 import BeautifulSoup
import re
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
}
import numpy as np

In [ ]:
%cd /content/drive/My Drive/Web_scraping/newlook/newlook_images

/content/drive/My Drive/Web_scraping/newlook/newlook_images


In [ ]:
def del_list( ctg , idx_list):
  for i in idx_list:
    os.remove('0/' + ctg + str(i) + '_0')
    os.remove('1/' + ctg + str(i) + '_1')
    os.remove('2/' + ctg + str(i) + '_2')
def delete_imgs_in_range( ctg , start , end):
  for i in range(start,end+1):
    if os.path.exists('0/'+ctg + str(i) + '_0' +'.jpg'):
      os.remove('0/' + ctg + str(i) + '_0' + '.jpg')
    if os.path.exists( '1/' + ctg + str(i) + '_1' +'.jpg'):
      os.remove( '1/'  + ctg + str(i) + '_1' + '.jpg')
    if os.path.exists(  '2/' + ctg + str(i) + '_2' +'.jpg'):
      os.remove(  '2/' + ctg + str(i) + '_2' + '.jpg')  
# def delete_imgs(sure=False):
#   if sure:
#     for pic in os.listdir('/content/drive/My Drive/Web_scraping/Hackett/Hackett_images'):
#       if re.split(r'\.',pic)[1]=='html':
#         continue
#       else:
#         os.remove(pic)

In [ ]:
def func_description(url):
  split1 = re.split(r'/',url)[-3] + ' ' +  re.split(r'/',url)[-1]
  # split2 = re.split(r'\.',split1)[0]
  return split1

In [ ]:
def save(img_category,img_urls,description):
  global count
  if len(img_urls)<1 :
    return

  file_name = img_category + str(count) + '_0'
  with open('0/' + file_name + '.jpg','wb') as f:
    try:
      im = requests.get(img_urls[0], headers=HEADERS)
    except ConnectionError as err:
      time.sleep(10) 
      im = requests.get(url, headers=HEADERS)
    f.write(im.content)   

  file_name = img_category + str(count) + '_1'
  with open('1/' + file_name + '.jpg','wb') as f:
    try:
      im = requests.get(img_urls[1], headers=HEADERS)
    except ConnectionError as err:
      time.sleep(10) 
      im = requests.get(url, headers=HEADERS)
    f.write(im.content) 

  file_name = img_category + str(count) + '_2'
  with open('2/' + file_name + '.jpg','wb') as f:
    try:
      im = requests.get(img_urls[2], headers=HEADERS)
    except ConnectionError as err:
      time.sleep(10) 
      im = requests.get(url, headers=HEADERS)
    f.write(im.content) 

  print(img_category + str(count) + '    name is  ' + description)
  count=count+1

In [ ]:
def generate_image_urls(page_url):
  urls = []

  if re.search('pack|maternity|activewear|joggers|blouse',page_url):
    return urls
  cloth = 'https://media2.newlookassets.com/i/newlook/'   +  re.split('/p/',page_url)[1]  + 'M9'  +  re.split('/uk',re.split('/p/',page_url)[0])[1]  + '.jpg?strip=true&qlt=80&w=720'
  upper = 'https://media2.newlookassets.com/i/newlook/'   +  re.split('/p/',page_url)[1]  +    re.split('/uk',re.split('/p/',page_url)[0])[1]   +   '.jpg?strip=true&qlt=80&w=720'
  full_pose = 'https://media2.newlookassets.com/i/newlook/'   +  re.split('/p/',page_url)[1]  + 'M1'  +  re.split('/uk',re.split('/p/',page_url)[0])[1]  + '.jpg?strip=true&qlt=80&w=720'
  try:
    r = requests.get(cloth, headers=HEADERS)
  except ConnectionError as err:
    time.sleep(10) 
    r = requests.get(cloth, headers=HEADERS)
  try:
    r1 = requests.get(full_pose, headers=HEADERS)
  except ConnectionError as err:
    time.sleep(10) 
    r1 = requests.get(full_pose, headers=HEADERS)
  try:
    r2 = requests.get(upper, headers=HEADERS)
  except ConnectionError as err:
    time.sleep(10) 
    r2 = requests.get(upper, headers=HEADERS)

  if int(r.headers.get('content-length'))<10000 or int(r1.headers.get('content-length'))<10000 or int(r2.headers.get('content-length'))<10000:
    urls = []
  else:
    urls.append(cloth); urls.append(full_pose) ; urls.append(upper)

  return urls 

In [ ]:
def scrape(html_file , ctg):
  print('sraping_',ctg) ; urls = []
  soup = BeautifulSoup(html_file, 'html.parser')
  for tag in soup.find_all('a',class_='product-item__name link--nounderline ng-binding'):
    urls.append(  'https://www.newlook.com' + tag['href'])
 
  print('total_count_for_this_category_is ' ,len(urls),' but some images dont have either cloth image or model image thus they will be discraded')
  for url in urls:
    save( ctg ,       generate_image_urls(url)     ,   func_description(url) ) 
    # time.sleep(0.1)

In [ ]:
def scrape_this(ctg):
  file = os.listdir('html_files')[0]
  with open('html_files/' + file) as f:
    scrape(f,ctg)
  os.remove('html_files/' + file)

In [ ]:
# count = 1
# scrape_this('men_t-shirts_')

sraping_ men_t-shirts_
total_count_for_this_category_is  531  but some images dont have either cloth image or model image thus they will be discraded
men_t-shirts_1    name is  white-sprite-logo-t-shirt 693939410
men_t-shirts_2    name is  grey-marl-ribbed-jersey-muscle-fit-vest 678223707
men_t-shirts_3    name is  off-white-overdyed-well-being-logo-t-shirt 693938412
men_t-shirts_4    name is  white-florida-varsity-logo-t-shirt 693929610
men_t-shirts_5    name is  white-short-sleeve-street-fighter-logo-t-shirt 685473610
men_t-shirts_6    name is  olive-vertical-stripe-t-shirt 683261633
men_t-shirts_7    name is  lilac-washed-long-sleeve-pocket-t-shirt 682997553
men_t-shirts_8    name is  stone-washed-long-sleeve-pocket-t-shirt 682997516
men_t-shirts_9    name is  mustard-washed-seam-trim-oversized-t-shirt 682996587
men_t-shirts_10    name is  off-white-ribbed-fine-knit-muscle-fit-t-shirt 682765512
men_t-shirts_11    name is  light-purple-oversized-t-shirt 680353951
men_t-shirts_12    n

In [ ]:
# scrape_this('men_shirts_')

sraping_ men_shirts_
total_count_for_this_category_is  244  but some images dont have either cloth image or model image thus they will be discraded
men_shirts_417    name is  burgundy-poplin-long-sleeve-shirt 647584967
men_shirts_418    name is  dark-green-poplin-long-sleeve-shirt 647584938
men_shirts_419    name is  olive-stripe-oversized-shirt 684477033
men_shirts_420    name is  olive-relaxed-fit-shacket 682982433
men_shirts_421    name is  off-white-relaxed-fit-shacket 682982412
men_shirts_422    name is  pale-grey-relaxed-fit-shacket 682982402
men_shirts_423    name is  black-relaxed-fit-shacket 682982401
men_shirts_424    name is  white-check-double-pocket-shirt 681560019
men_shirts_425    name is  pink-leaf-print-short-sleeve-shirt 681540970
men_shirts_426    name is  olive-cord-shirt 681532833
men_shirts_427    name is  khaki-twill-oversized-pocket-front-shacket 681398734
men_shirts_428    name is  camel-twill-oversized-pocket-front-shacket 681398717
men_shirts_429    name is  

In [ ]:
# scrape_this('men_knitwear_')

sraping_ men_knitwear_
total_count_for_this_category_is  39  but some images dont have either cloth image or model image thus they will be discraded
men_knitwear_584    name is  off-white-ribbed-fine-knit-muscle-fit-t-shirt 682765512
men_knitwear_585    name is  navy-fine-knit-crew-jumper 658459541
men_knitwear_586    name is  black-fine-knit-crew-jumper 658459501
men_knitwear_587    name is  off-white-fine-knit-crew-jumper 689537912
men_knitwear_588    name is  pale-blue-waffle-knit-side-stripe-shorts 683507845
men_knitwear_589    name is  green-waffle-knit-side-stripe-shorts 683507835
men_knitwear_590    name is  pale-blue-waffle-knit-side-stripe-t-shirt 683439045
men_knitwear_591    name is  green-waffle-knit-side-stripe-t-shirt 683439035
men_knitwear_592    name is  navy-fine-knit-muscle-fit-polo-shirt 683142041
men_knitwear_593    name is  off-white-fine-knit-muscle-fit-polo-shirt 683142012
men_knitwear_594    name is  mid-pink-ribbed-fine-knit-muscle-fit-polo-shirt 682760073
men_

In [ ]:
# scrape_this('men_sweatshirts_')

sraping_ men_sweatshirts_
total_count_for_this_category_is  168  but some images dont have either cloth image or model image thus they will be discraded
men_sweatshirts_616    name is  navy-jersey-pocket-front-long-sleeve-hoodie 679492041
men_sweatshirts_617    name is  khaki-jersey-pocket-front-long-sleeve-hoodie 679492034
men_sweatshirts_618    name is  black-jersey-pocket-front-long-sleeve-hoodie 679492001
men_sweatshirts_619    name is  light-green-jersey-crew-neck-sweatshirt 674361631
men_sweatshirts_620    name is  black-rose-sketch-embroidered-sweatshirt 670563801
men_sweatshirts_621    name is  burgundy-jersey-long-sleeve-crew-neck-sweatshirt 679497867
men_sweatshirts_622    name is  navy-jersey-long-sleeve-crew-neck-sweatshirt 679497841
men_sweatshirts_623    name is  khaki-jersey-long-sleeve-crew-neck-sweatshirt 679497834
men_sweatshirts_624    name is  grey-marl-jersey-long-sleeve-crew-neck-sweatshirt 679497807
men_sweatshirts_625    name is  black-jersey-long-sleeve-crew-ne

In [ ]:
# scrape_this('men_jeans_')

sraping_ men_jeans_
total_count_for_this_category_is  127  but some images dont have either cloth image or model image thus they will be discraded
men_jeans_762    name is  bright-blue-side-chain-skinny-jeans 684754144
men_jeans_763    name is  blue-side-chain-slim-fit-jeans 684755340
men_jeans_764    name is  off-white-tapered-leg-jeans 677989712
men_jeans_765    name is  black-ripped-stretch-skinny-jeans 675612301
men_jeans_766    name is  blue-mid-wash-ripped-knee-skinny-stretch-jeans 674631140
men_jeans_767    name is  black-washed-ripped-jeans 674630101
men_jeans_768    name is  navy-rinse-wash-skinny-stretch-jeans 609042841
men_jeans_769    name is  navy-rinse-wash-skinny-jeans 609038241
men_jeans_770    name is  grey-light-wash-skinny-fit-jeans 687601703
men_jeans_771    name is  white-skinny-stretch-jeans 686984710
men_jeans_772    name is  bright-blue-vintage-wash-skinny-jeans 686983344
men_jeans_773    name is  dark-grey-acid-wash-skinny-jeans 686983203
men_jeans_774    name 

In [ ]:
# scrape_this('men_trousers_')

sraping_ men_trousers_
total_count_for_this_category_is  205  but some images dont have either cloth image or model image thus they will be discraded
men_trousers_852    name is  pale-blue-side-stripe-joggers 686219845
men_trousers_853    name is  cream-nlm-tape-side-joggers 683494913
men_trousers_854    name is  black-nlm-tape-side-joggers 683494901
men_trousers_855    name is  light-green-jersey-cargo-joggers 683455131
men_trousers_856    name is  black-jersey-cargo-joggers 683455101
men_trousers_857    name is  navy-pintuck-joggers 683446241
men_trousers_858    name is  burgundy-jersey-cuffed-leg-joggers 679504267
men_trousers_859    name is  navy-jersey-cuffed-leg-joggers 679504241
men_trousers_860    name is  khaki-jersey-cuffed-leg-joggers 679504234
men_trousers_861    name is  grey-marl-jersey-cuffed-leg-joggers 679504207
men_trousers_862    name is  black-jersey-cuffed-leg-joggers 679504201
men_trousers_863    name is  grey-relaxed-fit-joggers 679245305
men_trousers_864    name

In [ ]:
# scrape_this('women_jeans_')

sraping_ women_jeans_
total_count_for_this_category_is  325  but some images dont have either cloth image or model image thus they will be discraded
women_jeans_985    name is  blue-ripped-crop-high-waist-boyfriend-jeans 684165440
women_jeans_986    name is  off-white-high-waist-belted-jeans 673928412
women_jeans_987    name is  bright-blue-mid-rise-life-shape-emilee-jeggings 670145643
women_jeans_988    name is  petite-blue-ripped-high-waist-90s-adalae-wide-leg-jeans 690628545
women_jeans_989    name is  black-ripped-knee-high-waist-tori-mom-jeans 687665601
women_jeans_990    name is  bright-blue-ripped-high-waist-hallie-super-skinny-jeans 685832144
women_jeans_991    name is  black-ripped-high-waist-hallie-super-skinny-jeans 685832101
women_jeans_992    name is  bright-blue-high-waist-hallie-super-skinny-jeans 685823944
women_jeans_993    name is  blue-mid-rise-india-super-skinny-jeans 684183440
women_jeans_994    name is  black-mid-rise-india-super-skinny-jeans 684183401
women_jeans

In [ ]:
# scrape_this('women_trousers_')

sraping_ women_trousers_
total_count_for_this_category_is  364  but some images dont have either cloth image or model image thus they will be discraded
women_trousers_1265    name is  stone-belted-zip-waist-trousers 679428516
women_trousers_1266    name is  black-floral-wide-leg-trousers 693630709
women_trousers_1267    name is  bright-pink-scuba-split-front-flared-trousers 692455476
women_trousers_1268    name is  black-scuba-split-front-flared-trousers 692455401
women_trousers_1269    name is  pale-pink-wide-leg-trousers 692333772
women_trousers_1270    name is  red-floral-high-waist-wide-leg-trousers 691499969
women_trousers_1271    name is  black-animal-print-drawstring-wide-leg-trousers 691499809
women_trousers_1272    name is  khaki-ribbed-wide-leg-trousers 690892934
women_trousers_1273    name is  stone-ribbed-split-front-flared-trousers 690884716
women_trousers_1274    name is  stone-ribbed-crop-wide-leg-trousers 690884516
women_trousers_1275    name is  grey-ribbed-crop-wide-l

In [ ]:
# scrape_this('women_tops-shirts-blouses_')

sraping_ women_tops-shirts-blouses_
total_count_for_this_category_is  682  but some images dont have either cloth image or model image thus they will be discraded
women_tops-shirts-blouses_1431    name is  brown-leopard-print-peplum-tea-blouse 693232729
women_tops-shirts-blouses_1432    name is  white-ditsy-floral-shirred-puff-sleeve-blouse 692835619
women_tops-shirts-blouses_1433    name is  white-spot-ruched-front-peplum-blouse 691328319
women_tops-shirts-blouses_1434    name is  pink-floral-open-back-tea-blouse 690690279
women_tops-shirts-blouses_1435    name is  black-floral-shirred-tie-back-peplum-blouse 686786609
women_tops-shirts-blouses_1436    name is  light-green-floral-crepe-button-front-blouse 686353339
women_tops-shirts-blouses_1437    name is  white-broderie-frill-high-neck-sleeveless-blouse 686250310
women_tops-shirts-blouses_1438    name is  blue-floral-square-neck-tie-front-blouse 685472949
women_tops-shirts-blouses_1439    name is  white-ditsy-floral-square-neck-blous

In [ ]:
# scrape_this('women_tops-t-shirts_')

sraping_ women_tops-t-shirts_
total_count_for_this_category_is  714  but some images dont have either cloth image or model image thus they will be discraded
women_tops-t-shirts_1915    name is  pale-yellow-honey-bee-logo-t-shirt 689629688
women_tops-t-shirts_1916    name is  red-positive-vibes-logo-t-shirt 689335160
women_tops-t-shirts_1917    name is  yellow-leopard-print-peplum-t-shirt 655450089
women_tops-t-shirts_1918    name is  white-toucan-do-it-logo-t-shirt 698335310
women_tops-t-shirts_1919    name is  white-sketch-heels-new-york-logo-t-shirt 697765010
women_tops-t-shirts_1920    name is  black-dogtooth-bow-t-shirt 697762601
women_tops-t-shirts_1921    name is  red-gingham-crinkle-crop-t-shirt 696284969
women_tops-t-shirts_1922    name is  red-cross-tie-back-crop-t-shirt 695283660
women_tops-t-shirts_1923    name is  black-cross-tie-back-crop-t-shirt 695283601
women_tops-t-shirts_1924    name is  black-amour-diamante-logo-t-shirt 694990601
women_tops-t-shirts_1925    name is  